# Предобработка данных. Проектная работа. Исследование надежности заемщиков.

## Описание проекта. 
Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [ ]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')

In [ ]:
try:
    data = pd.read_csv('data.csv')
except:    
    data = pd.read_csv('/datasets/data.csv')

In [ ]:
data.info()

In [ ]:
data.head(30)

In [ ]:
data.describe()

**<font color="maroon"></font>В таблице с данными 12 столбцов:</font>**

* children — количество детей в семье 
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

**<font color="maroon"></font>Имеются пропуски, дубликаты, неверный тип данных, отрицательные значения в днях работы.</font>**

### Шаг 2.1 Обработка пропусков.

In [ ]:
data.isna().sum()

In [ ]:
data.isna().mean()

**<font color="maroon"></font>Обнаружены пропуски в столбце о стаже работы и в столбце о размере доходов,в обоих столбцах доля пропусков составляет по 10%. Это достаточно значимо для анализа и порсто удалить эти строки нельзя. Пропуски в этих данных могут быть обусловлены нежеланием заемщиков сообщать свои данные, а могут отсутствовать, ввиду отсутствия у заёмщиков стажа работы и доходов, то есть равны нулю.</font>** 

**<font color="maroon"></font>Видим, что количество пропусков в этих столбцах совпадает и предполагаем, что это не отсутствие  существующей информации, а отсустствие стажа работы и, как следствие - доходов, то есть это категория безработных или студентов.</font>** 

**<font color="maroon"></font>Посмотрим, каким типам занятости соответствуют пропущенные значения.</font>**

In [ ]:
data['income_type'][data['total_income'].isnull()].value_counts().plot.bar();

**<font color="maroon"></font>Пропуски в данных есть у заемщиков с различными типапи занятости, можно сделать вывод, что пропуски обусловлены нежеланием заёмщиков их сообщать.</font>**

**<font color="maroon"></font>Поэтому для корректного анализа отсутствие этих данных лучше заменить на медианные значения.</font>**


In [ ]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

**<font color="maroon"></font>Проверим, не осталось ли пропусков</font>**

In [ ]:
data.isna().sum()

**<font color="maroon"></font>Итак, пропусков в данных не осталось, можем переходить к следующему шагу.</font>**

### Шаг 2.2 Проверка данных на аномалии и исправления.

**<font color="maroon"></font>Проверим данные на наличие отрицательных значений</font>**

In [ ]:
columns_numeric = data.dtypes[data.dtypes!='object'].index

In [ ]:
data_is_negative = data[columns_numeric]<0

In [ ]:
data_is_negative.sum()

In [ ]:
data['children'].value_counts()

**<font color="maroon"></font>Видим отрицательные значения в днях работы и количестве детей, так же есть явный выброс в виде 20 детей у 76 заёмщиков это может быть вызвано технической ошибкой или опечаткой, так как количество дней не используется для анализа, а аномальные значения в данных о детях представляют не значительную часть и подавляющее число детей у заемщиков равно нулю, заменим эти данные нулями.</font>**

In [ ]:
data.loc[data['children']<0, 'children'] = 0

In [ ]:
data.loc[data['children']==20, 'children'] = 0

In [ ]:
data.loc[data['days_employed']<0, 'days_employed'] = 0

In [ ]:
data['children'].value_counts()

In [ ]:
data['days_employed'].value_counts()

### Шаг 2.3. Изменение типов данных.

In [ ]:
data.info()

**<font color="maroon"></font>Вызвав информацию о типах данных, видим, что столбцы 'days_employed' и 'total_income' содержат вещественные числа.</font>**

**<font color="maroon"></font>Для удобства работы с данными заменим тип данных в столбце total_income на целочисленный.</font>**

In [ ]:
data['total_income'] = data['total_income'].astype('int')

In [ ]:
data.info()

### Шаг 2.4. Удаление дубликатов.

In [ ]:
data[data.duplicated()]

**<font color="maroon"></font>Приведём все данные типа 'object' к нижнему регистру и удалим явные дубликаты.</font>**

In [ ]:
str_columns = data.dtypes[data.dtypes=='object'].index
for index in str_columns:
    data[index] = data[index].str.lower()

In [ ]:
data = data.drop_duplicates()

data[data.duplicated()]

### Шаг 2.6. Категоризация дохода.

In [ ]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    elif 30000 < total_income <= 50000:
        return 'D'
    elif 50000 < total_income <= 200000:
        return 'C'
    elif 200000 < total_income <= 1000000:
        return 'B'
    else:
        return 'A'
data['total_income_category'] = data['total_income'].apply(total_income_category)
data['total_income_category'].value_counts()    

**<font color="maroon"></font>Для упрощения визуализации, разбила заёмщиков на категории по уровню дохода.</font>**

### Шаг 2.7. Категоризация целей кредита.

In [ ]:
data['purpose'].value_counts()

In [ ]:
def purpose_category(purpose):
    purpose_auto = ['на покупку своего автомобиля', 'автомобиль', 'сделка с подержанным автомобилем', 'автомобили', 'на покупку подержанного автомобиля', 'свой автомобиль','на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем']
    purpose_property = ['операции с недвижимостью', 'покупка коммерческой недвижимости', 'операции с жильем', 'покупка жилья для сдачи', 'операции с коммерческой недвижимостью','покупка жилья', 'жилье', 'покупка жилья для семьи', 'строительство собственной недвижимости', 'недвижимость', 'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'строительство недвижимости', 'ремонт жилью', 'покупка жилой недвижимости']
    purpose_wedding = ['свадьба', 'на проведение свадьбы', 'сыграть свадьбу', 'образование', 'получение дополнительного образования', 'получение образования', 'профильное образование', 'получение высшего образования', 'заняться образованием']
    purpose_education = ['заняться высшим образованием', 'дополнительное образование', 'высшее образование']
    if purpose in purpose_auto:
        return 'операции с автомобилем'
    elif purpose in purpose_property:
        return 'операции с недвижимостью'
    elif purpose in purpose_wedding:
        return 'проведение свадьбы'
    elif purpose in purpose_education:
        return 'получение образования'
    else:
        return 'не определено'
data['purpose_category'] = data['purpose'].apply(purpose_category)
data['purpose_category'].value_counts()

**<font color="maroon"></font>Скомпоновала цели кредитов по категориям.</font>**

**<font color="maroon"></font>Сгруппируем должников по количеству детей, по семейному статусу, по доходу и по целям кредита, посмотрим наглядно, как распределяются должники.</font>**

In [ ]:
children_debt = data.groupby(['children'])['debt'].value_counts().unstack().T/data['children'].value_counts()
children_debt.T[1].plot();

In [ ]:
family_status_debt = data.groupby(['family_status'])['debt'].value_counts().unstack().T/data['family_status'].value_counts()
family_status_debt.T[1].plot();

In [ ]:
total_income_debt = data.groupby(['total_income_category'])['debt'].value_counts().unstack().T/data['total_income_category'].value_counts()
total_income_debt.T[1].plot();

In [ ]:
purpose_debt = data.groupby(['purpose_category'])['debt'].value_counts().unstack().T/data['purpose_category'].value_counts()
purpose_debt.T[1].plot();

In [ ]:
purpose_debt_pivot = data.pivot_table(index='purpose_category', values='debt')

purpose_debt_pivot

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

**<font color="maroon"></font>Да, зависимость наблюдается.</font>**

##### Вывод 1:

**<font color="maroon"></font>У заемщиков без детей возвращаемость кредитов выше.</font>**

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

**<font color="maroon"></font>Да, зависимость наблюдается.</font>**

##### Вывод 2: 

**<font color="maroon"></font>Самая низкая вероятность должников среди вдовцов, чуть выше у - разведённых, еще выше - у тех, кто в браке и самая высокая - среди заемщиков в гражданском браке и не состоящих в браке.</font>**

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**<font color="maroon"></font>Да, зависимость наблюдается.</font>**

##### Вывод 3: 

**<font color="maroon"></font>Самая низкая вероятность должников среди заемщиков чей доход в диапазоне 30000-50000, чуть выше у тех, кто зарабатывает от 200000 до 1000000, еще выше вероятность появления долга у тех, чей доход выше 1000000 и от  50000 до 200000, самая высокая - у тех чей доход до 30000.</font>**

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

**<font color="maroon"></font>Наблюдается зависимось возврата кредита от его цели .</font>**

##### Вывод 4: 

**<font color="maroon"></font>Самая низкая вероятность должников, среди тех, чья цель кредита операции с недвижимостью, чуть выше - образование и свадьба, самая высокая вероятность неплательщиков среди тех, кто берёт деньги на автомобиль.</font>**

## Общий вывод:

**<font color="maroon"></font>Можно составить портрет самых добросовестных заёмщиков, ими будут вдовцы без детей, зарабатывающие 30000-50000, берущие кредит для операций с недвижимостью. Самые проблемные заемщики с детьми, не состоящие в браке, с доходом до 30000, занимающие деньги на операции с автомобилем.</font>**